In [1]:
import benepar, spacy
import nltk
benepar.download('benepar_en3')

[nltk_data] Downloading package benepar_en3 to
[nltk_data]     /home/coraline/nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


True

In [1]:
import datasets

In [2]:
imdb_dataset = datasets.load_dataset("imdb", split=['test'])

Reusing dataset imdb (/home/coraline/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
imdb_dataset['text']

TypeError: list indices must be integers or slices, not str

In [2]:
spacy.require_gpu()
nlp = spacy.load('en_core_web_md')
nlp.add_pipe('benepar', config={'model': 'benepar_en3'})

In [3]:
doc = nlp('The time for action is now. It is never too late to do something.')
sent = list(doc.sents)[0]
print(sent._.parse_string)

(S (NP (NP (DT The) (NN time)) (PP (IN for) (NP (NN action)))) (VP (VBZ is) (ADVP (RB now))) (. .))


/home/coraline/anaconda3/envs/bert-attack/lib/python3.7/site-packages/torch/distributions/distribution.py:46: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  'with `validate_args=False` to turn off validation.')


In [4]:
print(sent._.labels)

('S',)


In [5]:
print(list(sent._.children)[0])

The time for action


In [6]:
import nltk
from nltk import Tree, ParentedTree
parse_tree = ParentedTree.fromstring('(' + sent._.parse_string + ')')
print(parse_tree.pretty_print())

                                                
                           |                     
                           S                    
               ____________|__________________   
              NP                     |        | 
      ________|_______               |        |  
     |                PP             VP       | 
     |             ___|____       ___|___     |  
     NP           |        NP    |      ADVP  | 
  ___|___         |        |     |       |    |  
 DT      NN       IN       NN   VBZ      RB   . 
 |       |        |        |     |       |    |  
The     time     for     action  is     now   . 

None


In [1]:
import stanza
stanza.download('en') # download English model

2022-03-02 01:36:36 INFO: Downloading default packages for language: en (English)...


2022-03-02 01:37:25 INFO: Finished downloading models and saved to /home/coraline/stanza_resources.


In [16]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,pos,constituency')
doc = nlp('This is a test')
for sentence in doc.sentences:
    print(sentence.constituency)
    const_tree = str(sentence.constituency)

2022-03-02 01:56:44 INFO: Loading these models for language: en (English):
| Processor    | Package  |
---------------------------
| tokenize     | combined |
| pos          | combined |
| constituency | wsj      |

2022-03-02 01:56:44 INFO: Use device: gpu
2022-03-02 01:56:44 INFO: Loading: tokenize
2022-03-02 01:56:44 INFO: Loading: pos
2022-03-02 01:56:44 INFO: Loading: constituency
2022-03-02 01:56:44 INFO: Done loading processors!


(ROOT (S (NP (DT This)) (VP (VBZ is) (NP (DT a) (NN test)))))


In [17]:
import nltk
from nltk import Tree, ParentedTree

parse_tree = ParentedTree.fromstring(const_tree)
print(parse_tree.pretty_print())

         ROOT             
          |                
          S               
  ________|____            
 |             VP         
 |     ________|___        
 NP   |            NP     
 |    |         ___|___    
 DT  VBZ       DT      NN 
 |    |        |       |   
This  is       a      test

None


In [6]:
tree = doc.sentences[0].constituency
tree.label

'ROOT'

In [7]:
tree.children

[(S (NP (DT This)) (VP (VBZ is) (NP (DT a) (NN test))))]

In [8]:
tree.children[0].children

[(NP (DT This)), (VP (VBZ is) (NP (DT a) (NN test)))]

## PCFG
Extract Grammar from Treebank

In [20]:
nltk.download('treebank')
from nltk.corpus import treebank
from nltk import treetransforms
from nltk import induce_pcfg
from nltk.parse import pchart

[nltk_data] Downloading package treebank to
[nltk_data]     /home/coraline/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


In [22]:
# extract productions from three trees and induce the PCFG
print("Induce PCFG grammar from treebank data:")

productions = []
for item in treebank.fileids()[:2]:
    for tree in treebank.parsed_sents(item):
        # perform optional tree transformations, e.g.:
        tree.collapse_unary(collapsePOS = False)# Remove branches A-B-C into A-B+C
        tree.chomsky_normal_form(horzMarkov = 2)# Remove A->(B,C,D) into A->B,C+D->D
        productions += tree.productions()

Induce PCFG grammar from treebank data:


In [23]:
from nltk import Nonterminal
S = Nonterminal('S')
grammar = induce_pcfg(S, productions)
print(grammar)

Grammar with 86 productions (start state = S)
    S -> NP-SBJ S|<VP-.> [0.5]
    NP-SBJ -> NP NP-SBJ|<,-ADJP> [0.333333]
    NP -> NNP NNP [0.2]
    NNP -> 'Pierre' [0.0714286]
    NNP -> 'Vinken' [0.142857]
    NP-SBJ|<,-ADJP> -> , NP-SBJ|<ADJP-,> [1.0]
    , -> ',' [1.0]
    NP-SBJ|<ADJP-,> -> ADJP , [1.0]
    ADJP -> NP JJ [1.0]
    NP -> CD NNS [0.133333]
    CD -> '61' [0.333333]
    NNS -> 'years' [1.0]
    JJ -> 'old' [0.285714]
    S|<VP-.> -> VP . [1.0]
    VP -> MD VP [0.2]
    MD -> 'will' [1.0]
    VP -> VB VP|<NP-PP-CLR> [0.2]
    VB -> 'join' [1.0]
    VP|<NP-PP-CLR> -> NP VP|<PP-CLR-NP-TMP> [1.0]
    NP -> DT NN [0.0666667]
    DT -> 'the' [0.4]
    NN -> 'board' [0.142857]
    VP|<PP-CLR-NP-TMP> -> PP-CLR NP-TMP [1.0]
    PP-CLR -> IN NP [1.0]
    IN -> 'as' [0.25]
    NP -> DT NP|<JJ-NN> [0.133333]
    DT -> 'a' [0.4]
    NP|<JJ-NN> -> JJ NN [1.0]
    JJ -> 'nonexecutive' [0.285714]
    NN -> 'director' [0.285714]
    NP-TMP -> NNP CD [1.0]
    NNP -> 'Nov.' [0.0714286

## Semantic Role Labeling

In [1]:
from allennlp.predictors.predictor import Predictor
from spacy.language import Language
from spacy.tokens import Doc

In [2]:
import benepar, spacy
import nltk
benepar.download('benepar_en3')

[nltk_data] Downloading package benepar_en3 to
[nltk_data]     /home/coraline/nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


True

In [6]:
class SRLComponent:

    def __init__(self, nlp: Language, model_path: str):
        if not Doc.has_extension("srl"):
            Doc.set_extension("srl", default=None)
        
        self.predictor = Predictor.from_path(model_path)

    def __call__(self, doc: Doc):
        predictions = self.predictor.predict(sentence=doc.text)
        doc._.srl = predictions
        return doc

In [4]:
@Language.factory("srl", default_config={
    "model_path": "https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz"})
def create_srl_component(nlp: Language, name: str, model_path: str):
    return SRLComponent(nlp, model_path)

In [7]:
#nlp = spacy.blank('en')
nlp = spacy.load('en_core_web_md')
nlp.add_pipe('benepar', config={'model': 'benepar_en3'})
nlp.add_pipe("srl")

doc = nlp("The dog trashed the apartment in under 30 seconds. I am devastated.")

/home/coraline/anaconda3/envs/bert-attack/lib/python3.7/site-packages/spacy/util.py:1504: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
2022-03-02 06:11:08,869 - INFO - allennlp.common.plugins - Plugin allennlp_models available
2022-03-02 06:11:09,272 - INFO - cached_path - cache of https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz is up-to-date
2022-03-02 06:11:09,272 - INFO - allennlp.models.archival - loading archive file https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz from cache at /home/coraline/.allennlp/cache/b5f1db011cc85691a5fa2bf29e055a712261a2e5d74a74edd7da2fffc98d4ab8.4c4ac7e06ec3d85631bd26b839f90b5a375d3ceeb43e3c74f1c

2022-03-02 06:11:25,558 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.attention.self.value.bias
2022-03-02 06:11:25,559 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.attention.self.value.weight
2022-03-02 06:11:25,559 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.intermediate.dense.bias
2022-03-02 06:11:25,559 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.intermediate.dense.weight
2022-03-02 06:11:25,559 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.output.LayerNorm.bias
2022-03-02 06:11:25,560 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.output.LayerNorm.weight
2022-03-02 06:11:25,560 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.output.dense.bias
2022-03-02 06:11:25,560 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.output.dense.weight
2022-03-02 06:11:25,560 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.10.a

2022-03-02 06:11:25,581 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.3.output.dense.weight
2022-03-02 06:11:25,581 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.4.attention.output.LayerNorm.bias
2022-03-02 06:11:25,583 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.4.attention.output.LayerNorm.weight
2022-03-02 06:11:25,583 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.4.attention.output.dense.bias
2022-03-02 06:11:25,584 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.4.attention.output.dense.weight
2022-03-02 06:11:25,584 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.4.attention.self.key.bias
2022-03-02 06:11:25,584 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.4.attention.self.key.weight
2022-03-02 06:11:25,584 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.4.attention.self.query.bias
2022-03-02 06:11:25,585 - INFO - allennlp.nn.initializers - 

2022-03-02 06:11:25,605 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.8.attention.self.query.bias
2022-03-02 06:11:25,605 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.8.attention.self.query.weight
2022-03-02 06:11:25,606 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.8.attention.self.value.bias
2022-03-02 06:11:25,606 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.8.attention.self.value.weight
2022-03-02 06:11:25,608 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.8.intermediate.dense.bias
2022-03-02 06:11:25,608 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.8.intermediate.dense.weight
2022-03-02 06:11:25,608 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.8.output.LayerNorm.bias
2022-03-02 06:11:25,608 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.8.output.LayerNorm.weight
2022-03-02 06:11:25,609 - INFO - allennlp.nn.initializers -    bert_model.en

In [8]:
from nltk import Tree, ParentedTree

#sent = list(doc.sents)[0]
#parse_tree = ParentedTree.fromstring('(' + doc._.parse_string + ')')
print(doc._.srl)
#print(parse_tree.pretty_print())

{'verbs': [{'verb': 'trashed', 'description': '[ARG0: The dog] [V: trashed] [ARG1: the apartment] [ARGM-TMP: in under 30 seconds] . I am devastated .', 'tags': ['B-ARG0', 'I-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARGM-TMP', 'I-ARGM-TMP', 'I-ARGM-TMP', 'I-ARGM-TMP', 'O', 'O', 'O', 'O', 'O']}, {'verb': 'am', 'description': 'The dog trashed the apartment in under 30 seconds . [ARG1: I] [V: am] [ARG2: devastated] .', 'tags': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ARG1', 'B-V', 'B-ARG2', 'O']}, {'verb': 'devastated', 'description': 'The dog trashed the apartment in under 30 seconds . [ARG1: I] am [V: devastated] .', 'tags': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ARG1', 'O', 'B-V', 'O']}], 'words': ['The', 'dog', 'trashed', 'the', 'apartment', 'in', 'under', '30', 'seconds', '.', 'I', 'am', 'devastated', '.']}
